# Modos optimizados

## Modo de solo lectura

A veces, necesitarás abrir o escribir archivos `.xlsx` extremadamente grandes, y las rutinas comunes en openpyxl no podrán manejar esa carga. Afortunadamente, hay dos modos que le permiten leer y escribir cantidades ilimitadas de datos con un consumo de memoria (casi) constante.

Presentando `openpyxl.worksheet._read_only.ReadOnlyWorksheet`:

In [2]:
from openpyxl import load_workbook
wb = load_workbook(filename='ejemplo2.xlsx', read_only=True) # Cargamos el workbook en modo lectura
ws = wb['google_mx_slime_phrase-match_20'] # Seleccionamos la hoja

# Mostramos todos los valore de la hoja
for row in ws.rows:
    for cell in row:
        print(cell.value)

# Close the workbook after reading
wb.close()

=SUM(1, 1)
Keyword
Country
Difficulty
Volume
CPC
Clicks
CPS
Return Rate
Parent Keyword
Last Update
SERP Features
Global volume
Organic Clicks Only
Paid Clicks Only
Paid & Organic Clicks
Searches Without Clicks
1
slime
mx
2
50000
0.25
21135
0.42
1.4
videos de como hacer slime
2021-05-18 00:10:06
People also ask,Top stories,Thumbnails,Image pack
1070000
0.37
0.01
0.01
0.61
2
slime rancher
mx
2
26000
0.8
25239
0.97
1.44
slime rancher
2021-05-14 15:56:20
Sitelinks,People also ask,Top stories,Videos
439000
0.64
0.02
0.02
0.32
3
tensei shitara slime datta ken
mx
3
24000
0
11642
0.49
1.79
tensei shitara slime datta ken
2021-05-14 13:30:20
Sitelinks,People also ask
541000
0.4
0
0
0.6
4
como hacer slime
mx
2
17000
0.05
11000
0.65
1.46
slime casero
2021-05-14 18:28:30
Featured snippet,People also ask,Top stories,Videos,Image pack
56000
0.55
0.01
0
0.44
5
super slime sam
mx
0
16000
None
13496
0.84
2.27
super slime sam
2021-05-15 00:05:02
Top stories,Videos,People also ask,Image pack,Sitelinks
530

**ATENCIÓN:**
- `openpyxl.worksheet._read_only.ReadOnlyWorksheet` es de solo lectura
- A diferencia de un libro de trabajo normal, un libro de trabajo de solo lectura utilizará la carga diferida. El libro de trabajo debe cerrarse explícitamente con el método `close()`.

Las celdas devueltas no son `openpyxl.cell.cell.Cell` normales, sino `openpyxl.cell._read_only.ReadOnlyCell`.

### Dimensiones de la hoja de trabajo

El modo de solo lectura se basa en las aplicaciones y bibliotecas que crearon el archivo y proporcionan información correcta sobre las hojas de trabajo, específicamente la parte utilizada de la misma, conocida como dimensiones. Algunas aplicaciones configuran esto incorrectamente. Puede verificar las dimensiones aparentes de una hoja de trabajo usando `ws.calculate_dimension()`. Si esto devuelve un rango que sabe que es incorrecto, como A1: A1, entonces simplemente restablecer los atributos `max_row` y `max_column` debería permitirle trabajar con el archivo:

In [4]:
ws.reset_dimensions()

## Modo de solo escritura

Aquí nuevamente, el `openpyxl.worksheet.worksheet.Worksheet` regular ha sido reemplazado por una alternativa más rápida, el `openpyxl.worksheet._write_only.WriteOnlyWorksheet`. Cuando desee volcar grandes cantidades de datos, asegúrese de tener `lxml` instalado (`pip install lxml`).

In [5]:
from openpyxl import Workbook

wb = Workbook(write_only=True) # Creamos un nuebo WB en modo escritura
ws = wb.create_sheet() # Creamos una nueva hoja

# Vamos a llenarlo con 100 filas y 200 columnas
for row in range(100):
    ws.append(['%d' % i for i in range(200)])

# Guardamos el archivo
wb.save('ejemplo3.xlsx')

Si deseas tener celdas con estilos o comentarios, usa `openpyxl.cell.WriteOnlyCell()`

In [6]:
from openpyxl.cell import WriteOnlyCell
from openpyxl.comments import Comment
from openpyxl.styles import Font

wb = Workbook(write_only = True) # Creamos un nuevo WB en modo escritura
ws = wb.create_sheet() # Creamos una nueva hoja


cell = WriteOnlyCell(ws, value="hello world") # Guardamos en una variable el texto a escribir
cell.font = Font(name='Courier', size=36) # Elegimos la fuente y el tamaño
cell.comment = Comment(text="Un comentario", author="Nacho") # Escribimos un comentario
ws.append([cell, 3.14, None]) # 3 celdas: una con el texto y el comentario, un numero y una vacía
wb.save('ejemplo4.xlsx')

Esto creará un libro de trabajo de solo escritura con una sola hoja y agregará una fila de 3 celdas: una celda de texto con una fuente personalizada y un comentario, un número de punto flotante y una celda vacía (que se descartará de todos modos).

**ATENCIÓN:**
- A diferencia de un libro de trabajo normal, un libro de trabajo de solo escritura recién creado no contiene ninguna hoja de trabajo; una hoja de trabajo debe crearse específicamente con el método `create_sheet()`.
- En un libro de trabajo de solo escritura, las filas solo se pueden agregar con `append()`. No es posible escribir (o leer) celdas en ubicaciones arbitrarias con `cell()` o `iter_rows()`.
- Es capaz de exportar una cantidad ilimitada de datos (incluso más de los que Excel puede manejar en realidad), mientras mantiene el uso de memoria por debajo de 10Mb.
- Un libro de trabajo de solo escritura solo se puede guardar una vez. Después de eso, cada intento de guardar el libro de trabajo o agregar () a una hoja de trabajo existente generará una excepción `openpyxl.utils.exceptions.WorkbookAlreadySaved`.
- Todo lo que aparece en el archivo antes de los datos de celda reales debe crearse antes de agregar las celdas porque debe escribirse en el archivo antes de esa fecha. Por ejemplo, *freeze_panes* debe configurarse antes de agregar celdas.